In [1]:
import os
import pandas as pd 
import shutil
import configparser

In [2]:
def getBandsIndexes(cat,bands):
    return [list(cat.columns).index(c)+1 for c in bands]

In [3]:
def makeConfig(dirCat:str,fileNameCat:str,idcolumn:str,zspeccolumn:str,
               features:list,expNum:int,dirConfigs:str,configName:str):
    
    cat=pd.read_csv(os.path.join(dirCat,fileNameCat))
    headers=list(cat)
    idcolumnidx=str(int(headers.index('Seq')+1))
    zspeccolumnidx=str(int(headers.index(zspeccolumn)+1))
    featuresIdx=getBandsIndexes(cat,features)
    featuresIdxStr=','.join(str(x) for x in featuresIdx)
    nNeuronsLayer1=str(int(2*len(featuresIdx)+1))
    nNeuronsLayer2=str(int(len(featuresIdx)-1))

    newIni='exp'+str(expNum).zfill(3)+'_'+configName+'.ini'
    shutil.copyfile(os.path.join(dirConfigs,fileConfigExample),os.path.join(dirConfigs,newIni))
    parser=configparser.RawConfigParser()
    parser.read(os.path.join(dirConfigs,newIni))
    parser.set('Train Input Info','photoColumnIdxList',featuresIdxStr)
    parser.set('Train Input Info','dirPath',dirCat)
    parser.set('Train Input Info','idcolumnidx',idcolumnidx)
    parser.set('Train Input Info','zspeccolumnidx',zspeccolumnidx)
    parser.set('Train Input Info','name',fileNameCat)
    parser.set('Test Input Info','photoColumnIdxList',featuresIdxStr)
    parser.set('Test Input Info','name',fileNameCat)
    parser.set('Test Input Info','dirPath',dirCat)
    parser.set('Test Input Info','idcolumnidx',idcolumnidx)
    parser.set('Test Input Info','zspeccolumnidx',zspeccolumnidx)
    parser.set('Output Info','dirpath',
               'Sources\COSMOS_photoZ\COSMOS_v3.0\experimentsArticle\Vis_UV_IR\exp'+str(expNum).zfill(3))
    parser.set('Output Info','modelSubDirName','ex'+str(expNum).zfill(3))
    parser.set('Learning Info','nHiddenNeurons',nNeuronsLayer1+','+nNeuronsLayer2)
    parser.set('Train Output Info','nameSuffix','_'+fileNameCat)
    parser.set('Test Output Info','nameSuffix','_'+fileNameCat)
    parser.set('Log Info','nameSuffix','_'+fileNameCat)
    parser.set('Restart Info','nameSuffix','_'+fileNameCat)

    with open(os.path.join(dirConfigs,newIni), 'w') as configfile:
        parser.write(configfile)
    return

In [4]:
dirCat=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\catalogs\forPaper\v05'
dirConfigs=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR'
fileConfigExample=r'exXX_phz_z_ConfigExample.ini'
expNum=1
fileNameCat=r'05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors.csv'
idcolumn='Seq'
zspeccolumn='z_spec'
cat=pd.read_csv(os.path.join(dirCat,fileNameCat))

In [5]:
bands={'Vis':['B','V','r','ip','zpp'],
      'UV':['u'],
      'IR':['Y','J','H','Ks']}

In [6]:
for key,val in bands.items():
    blist=[b+'magap3' for b in val]
    bands[key]=blist

In [24]:
for key,val in bands.items():
    fset=val
    fsetName=key
    makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('.csv','')+'_'+fsetName)
    expNum+=1 


fsetName='Vis_UV'
fset=bands['Vis']+bands['UV']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName)
expNum+=1 

fsetName='Vis_IR'
fset=bands['Vis']+bands['IR']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName)
expNum+=1 

fsetName='Vis_UV_IR'
fset=bands['Vis']+bands['UV']+bands['IR']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName)
expNum+=1 

In [28]:
os.chdir('D:\\')

# 19.01.2021. Vis-UV-IR experiments
configs=os.listdir(dirConfigs)
configs=[conf for conf in configs if conf.startswith('exp0')]
for conf in configs:
    conf=os.path.join(dirConfigs,conf)
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp001_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp002_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_UV.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp003_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_IR.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp004_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp005_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_IR.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp006_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_IR.ini
Ready


In [8]:
### Identifying most important IR band
expNum=7

In [9]:
for band in bands['IR']:
    fsetName='Vis_UV_'+band
    fset=bands['Vis']+bands['UV']+[band]
    makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('.csv','')+'_'+fsetName)
    expNum+=1 

In [10]:
os.chdir('D:\\')

configs=os.listdir(dirConfigs)
confIds=['exp007','exp008','exp009','exp010']
for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp007_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_Ymagap3.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp008_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_Jmagap3.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp009_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_Hmagap3.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp010_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_Ksmagap3.ini
Ready


In [11]:
### With narrow bands

In [12]:
narrows=['IB574ap3','IA527ap3','IA624ap3','NB816ap3']
bandBasic=bands['Vis']+bands['UV']+['Hmagap3']

In [13]:
expNum

11

In [14]:
for band in narrows:
    fsetName='Vis_UV_'+band
    fset=bands['Vis']+bands['UV']+[band]
    makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('.csv','')+'_'+fsetName)
    expNum+=1 

In [ ]:
os.chdir('D:\\')

configs=os.listdir(dirConfigs)
confIds=['exp011','exp012','exp013','exp014']
for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp011_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_IB574ap3.ini
